# Open dataset and do some plotting!

This notebook will walk through opening the generated `combined.json` with xarray and using xarray + metpy + hvplot to create interactive plots.

First, import the needed libraries:
* Metpy - used for generating projection info from GOES data
* Cartopy - used for projecting/transforming coordinate systems
* fsspec - open remote files

In [ ]:
import hvplot.xarray
import matplotlib.pyplot as plt
import fsspec
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

### Login to EarthData and get S3 access credentials (2-hour expiration)

In [ ]:
import earthaccess
auth = earthaccess.login(persist=True)
creds = earthaccess.get_s3_credentials(daac='GES_DISC')

### Set up fsspec ReferenceFileSystem to use `./combined.json` and S3 credentials

In [ ]:
r_opts = {'anon':False,          
          'key':creds['accessKeyId'], 
          'secret':creds['secretAccessKey'], 
          'token':creds['sessionToken']}    #ncfiles

# Use this line for the example JSON
fs = fsspec.filesystem(
    'reference', 
    fo="./combined.json", 
    # fo="./example_jsons/combined.json", 
    remote_protocol='s3', 
    remote_options=r_opts,
    skip_instance_cache=True
)

### Filesystem mapper can be opened directly with xarray and the Zarr engine.

In [ ]:
%%time
ds = xr.open_dataset(fs.get_mapper(), engine='zarr', consolidated=False)
ds

### Use the xarray dataset "as usual" - only required data chunks will be pulled in

In [ ]:
ax = plt.subplot(projection=ccrs.Orthographic(central_latitude=44.9, central_longitude=-103.7))

ds.T2M.sel(time='2024-03-15T12:00:00', method='nearest').plot(
    transform=ccrs.PlateCarree(), 
    cmap='jet', 
    robust=True
)

ax.coastlines()
ax.add_feature(cfeature.BORDERS)

In [ ]:
ds.QV10M.hvplot(
    groupby='time', cmap='jet', 
    crs=ccrs.PlateCarree(), project=True,
    coastline=True, rasterize=True, features=['borders', 'states'],
    widget_type='scrubber', widget_location='bottom',
    clim=(1e-5, 2e-2)
)

In [ ]:
ds.T2M.sel(lat=[5, 20, 40, 60, 80], method='nearest').hvplot.kde('T2M', by='lat', alpha=0.5)